In [1]:
from dask.distributed import Client
Client()

Client Scheduler: tcp://127.0.0.1:53337 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [2]:
def transform(line):
    if "geometry" not in line:
        return None
    props = line["properties"]
    geom = line["geometry"]
    return {
        'ts': props["timestamp"],
        'alt': props["altitude"] if 'altitude' in props else None,
        'vacc': props["vertical_accuracy"] if 'vertical_accuracy' in props else None,
        'hacc': props["horizontal_accuracy"] if "horizontal_accuracy" in props else None,
        'wifi': props["wifi"],
        'batt': props["battery_level"] if "battery_level" in props else None,
        'mot': props["motion"] if "motion" in props else None,
        'speed': props["speed"] if "speed" in props else None,
        'lat': geom["coordinates"][1],
        'lon': geom["coordinates"][0],
    }

def load_data():
    import dask.bag as db
    import dask.dataframe as ddf
    import json
    import pandas as pd
    import datetime
    import numpy as np
#     jsons = db.read_text("dl/history/*.json").map(json.loads).map(transform).filter(lambda x: x is not None)
    jsons = db.read_text("dl/history/2019*.json").map(json.loads).map(transform).filter(lambda x: x is not None)
    meta = {
        'ts': str, #todo fix
        'alt': float, #int
        'vacc': float, #int
        'hacc': float, #int
        'wifi': str,
        'batt': float,
        'mot': str,
        'speed': float, #int
        'lat': float,
        'lon': float,
    }
    df = jsons.to_dataframe(meta=meta)
    df["ts"]= ddf.to_datetime(df["ts"]) 
    return df
    
df = load_data()
df.dtypes

ts       datetime64[ns]
alt             float64
vacc            float64
hacc            float64
wifi             object
batt            float64
mot              object
speed           float64
lat             float64
lon             float64
dtype: object

In [3]:
df.to_csv('output/out.csv', single_file=True)

['/Users/ncrocker/dev/personal/overland-lambda/rollup/output/out.csv']